# Alternating Least-Squares (ALS)

ALS is one of many techniques used for constructing recommendation engines. Two fundamental distinctions should be borne in mind: (1) item-based vs. user-based systems, and (2) memory-based vs. model-based systems.

For (1), consider two different strategies: (a) I recommend items to you that are similar to other items you've used/bought/read/watched; and (b) I recommend items to you that people similar to you have used/bought/read/watched. The first is the item-based strategy; the second is the user-based strategy. For more see [this Wikipedia article](https://en.wikipedia.org/wiki/Collaborative_filtering) and [this blog post](https://towardsdatascience.com/recommendation-systems-models-and-evaluation-84944a84fb8e). [This post](https://dataconomy.com/2015/03/an-introduction-to-recommendation-engines/) on dataconomy is also useful.

The core of (2) is in whether (a) the system uses existing ratings to compute user-user or item-item similarity, or (b) the system uses machine learning techniques to make predictions.

ALS is user-based and model-based, as should become clear below!

In [2]:
from random import gauss as gs
from random import uniform as uni
from random import seed
import numpy as np

Suppose we start with a matrix $R$ of users and products, where each cell records the ranking the relevant user gave to the relevant product. Very often we'll be able to record this data as a sparse matrix, because many users will not have ranked many items.


ALS recommendation systems are often implemented in Spark architectures because of the appropriateness for distributed computing. ALS systems often involve very large datasets (consider how much data the recommendation engine for NETFLIX must have, for example!), and it is often useful to store them as sparse matrices, which Spark's ML library can handle. In fact, Spark's mllib even includes a "Rating" datatype!

Imagine factoring this matrix into a user matrix and an item matrix: $R = PQ^T$. Then we could predict a user's ranking of a particular item simply by calculating $p^Tq$, the dot-product of a row of $P$ and a column of $Q^T$. Why? The user vector records the user's preferences with respect to certain latent features, while the item vector records how the item ranks with respect to those preferences.

We could therefore calculate *all* predictions, i.e. fill in the gaps in $R$, by solving for P and Q.

But how can we solve for two variables at once? Well, we can't. But here's what we can do:

Make guesses of the values for P and Q. Then hold the values of one *constant* so that we can optimize for the values of the other!

Basically this converts our problem into a familiar *least-squares* problem. See [this page](https://textbooks.math.gatech.edu/ila/least-squares.html) and [this page](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/) for more details, but here's the basic idea:

If we have an equation $Ax = b$ for *non-square* $A$, then we have:

$A^TAx = A^Tb$ <br/>
Thus: <br/>
$x = (A^TA)^{-1}A^Tb$

This $(A^TA)^{-1}A^T$ **is the pseudo-inverse of** $A$.

In [6]:
np.random.seed(42)

A = np.random.rand(5, 5)
b = np.random.rand(5, 1)

In [1]:
np.linalg.inv(A.T.dot(A)).dot(A.T).dot(b)

In [2]:
np.linalg.pinv(A).dot(b)

"When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our 'tastes' not being nearly as complex. Even if i’ve \[sic\] viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original 'all users by all items' matrix into something much smaller that represents 'all items by some taste dimensions' and 'all users by some taste dimensions'. These dimensions are called ***latent or hidden features*** and we learn them from our data" ([Medium article: "ALS Implicit Collaborative Filtering"](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)).

In [16]:
# If P = users and Q = items, then we want to approximate R = PQ^T
# Let's generate R.

seed(42)
ratings = []
for _ in range(100):
    user = []
    for _ in range(100):
        user.append(int(uni(1, 6)))
    ratings.append(user)

In [17]:
users = []

for _ in range(100):
    user = []
    for _ in range(20):
        user.append(gs(0, 2 / np.sqrt(3)))
    users.append(user)

In [18]:
items = []

for _ in range(100):
    item = []
    for _ in range(20):
        item.append(gs(0, 2 / np.sqrt(3)))
    items.append(item)

In [19]:
ratings_arr = np.asarray(ratings)
users_arr = np.asarray(users)
items_arr = np.asarray(items)

In [20]:
guess = users_arr.dot(items_arr.T)

In [21]:
err = (ratings_arr - guess)**2

In [3]:
np.sum(err)

In [18]:
# Users: m x n (m users)
# Items: r x n (r items)
# Ratings: m x r

In [23]:
def als(ratings, users, items, reps=10):
    
    ratings_cols = ratings.T
    for _ in range(reps):
        new_users = []
        for i in range(len(ratings)):
            user = np.linalg.pinv(items).dot(ratings[i])
            new_users.append(user)
        new_users = np.asarray(new_users)
        
        new_items = []
        for i in range(len(ratings)):
            item = np.linalg.pinv(new_users).dot(ratings_cols[i])
            new_items.append(item)
        new_items = np.asarray(new_items)
        
        guess = new_users.dot(new_items.T)
        err = (ratings - guess)**2
        print(np.sum(err))
        
        items = new_items
        
    return new_users, new_items

In [4]:
new_u, new_i = als(ratings_arr, users_arr, items_arr)